# Recalcular indicadores por buffer y para niveles de agregación superiores

In [35]:
import shapefile
from scripts import utils
from scripts import recalculate_indicators
import pandas as pd
from scripts.path_finders import get_weights_path
from scripts.create_indicators import get_or_create_indicators_df
from scripts.geo_utils import join_df_with_shp
from scripts.path_finders import get_division_path

## 1. Recalcular indicadores de radios censales para cada buffer

Los indicadores calculados por buffer son un promedio ponderado de los indicadores de radios censales que intersectan el buffer. Recalculamos los indicadores para todos los shapefiles generados con buffers de puntos de transporte.

In [36]:
# se omiten aquellos indicadores que son cuentas, y por lo tanto su agregación no es una media
omit_fields = ["hospitales", "educ_priv", "educ_pub"]
recalculate_indicators.main(skip=["14_10_2"], recalculate=False, area_level="RADIO", 
                            subcategory="buffers", omit_fields=omit_fields)

Calculating accesibilidad_en_estaciones_de_subte-buffer1000 indicators
Calculating accesibilidad_en_estaciones_de_subte-buffer1500 indicators
Calculating accesibilidad_en_estaciones_de_subte-buffer2000 indicators
Calculating accesibilidad_en_estaciones_de_subte-buffer300 indicators
Calculating accesibilidad_en_estaciones_de_subte-buffer500 indicators
Calculating accesibilidad_en_estaciones_de_subte-buffer750 indicators
Calculating estaciones_de_ferrocarril-buffer1000 indicators
Calculating estaciones_de_ferrocarril-buffer1500 indicators
Calculating estaciones_de_ferrocarril-buffer2000 indicators
Calculating estaciones_de_ferrocarril-buffer300 indicators
Calculating estaciones_de_ferrocarril-buffer500 indicators
Calculating estaciones_de_ferrocarril-buffer750 indicators
Calculating estaciones_de_metrobus-buffer1000 indicators
Calculating estaciones_de_metrobus-buffer1500 indicators
Calculating estaciones_de_metrobus-buffer2000 indicators
Calculating estaciones_de_metrobus-buffer300 indi

## 2. Calcular indicadores para niveles de agregación superiores

Algunos indicadores fueron calculados sólo para un nivel de agregación, pero podrían agregarse para los niveles subsiguientes (los indicadores calculados por radio censal, pueden agregarse por fracción, comuna o barrio). Antes de crear los shps finales, se recalcularán aquellos indicadores que sólo estén presentes en un nivel de agregación pero pueden estar en otros.

In [37]:
indicators_radio = get_or_create_indicators_df("RADIO")
indicators_frac = get_or_create_indicators_df("FRAC")
indicators_comuna = get_or_create_indicators_df("DPTO")

In [38]:
def get_id_frac(id_radio):
    comuna, frac, ra = id_radio.split("_")
    return "_".join([comuna.zfill(3), frac])

def get_id_comuna(id_radio):
    comuna, frac, ra = id_radio.split("_")
    return comuna

indicators_radio["CO_FRACC"] = map(get_id_frac, indicators_radio["CO_FRAC_RA"])
indicators_radio["COMUNA"] = map(get_id_comuna, indicators_radio["CO_FRAC_RA"])

In [39]:
dfr = indicators_radio.copy()

hab_frac = indicators_radio.groupby(by="CO_FRACC").sum()["hab"]
hab_co = indicators_radio.groupby(by="COMUNA").sum()["hab"]

In [40]:
print hab_frac.sum(), hab_co.sum()

2890151.0 2890151.0


In [41]:
dfr["hab_co"] = indicators_radio["COMUNA"].map(hab_co.to_dict())
dfr["hab_frac"] = indicators_radio["CO_FRACC"].map(hab_frac.to_dict())

In [42]:
print len(dfr)
total_hab1 = dfr["hab"].sum()
total_hab2 = dfr.groupby(by="CO_FRACC").sum()["hab"].sum()
total_hab3 = dfr.groupby(by="COMUNA").sum()["hab"].sum()
print total_hab1, total_hab2, total_hab3 

3553
2890151.0 2890151.0 2890151.0


In [43]:
dfr["pond_frac"] = dfr["hab"] / dfr["hab_frac"]
dfr["pond_co"] = dfr["hab"] / dfr["hab_co"]

In [44]:
print dfr.groupby("CO_FRACC").sum()["pond_frac"].sum()
print dfr.groupby("COMUNA").sum()["pond_co"].sum()

351.0
15.0


In [45]:
omit_fields = ["educ_priv", "educ_pub", "hospitales", "CO_FRAC_RA", "COMUNA", "CO_FRACC"]

# recalculamos indicadores de radio por fracción censal
for indic in indicators_radio.columns:
    if indic not in omit_fields and indic not in indicators_frac.columns:
        dfr["temp"] = dfr["pond_frac"] * dfr[indic]
        indic_dict = dfr.groupby(by="CO_FRACC").sum()["temp"].to_dict()
        indicators_frac[indic] = indicators_frac["CO_FRACC"].map(indic_dict) 

# recalculamos indicadores de radio por fracción censal
for indic in indicators_radio.columns:
    if indic not in omit_fields and indic not in indicators_comuna.columns:
        dfr["temp"] = dfr["pond_co"] * dfr[indic]
        indic_dict = dfr.groupby(by="COMUNA").sum()["temp"].to_dict()
        indicators_comuna[indic] = indicators_comuna["COMUNAS"].map(indic_dict) 

## 3. Crear shapefiles de los indicadores

In [46]:
join_df_with_shp(get_division_path("radios_censo_2010"), indicators_radio.set_index("CO_FRAC_RA"), "indicadores")
join_df_with_shp(get_division_path("fracciones_caba_censo_2010"), indicators_frac.set_index("CO_FRACC"), "indicadores")
join_df_with_shp(get_division_path("comunas_caba_censo_2010"), indicators_comuna.set_index("COMUNAS"), "indicadores")

## 4. Recalcular indicadores por barrio

Los barrios son unidades geográficas no construidas como conjuntos de radios o fracciones censales, de manera que el recálculo de indicadores para este nivel de agregación no se puede hacer con joins basados en un sistema de ids. Para este caso, utilizamos un procedimiento similar al del recálculo de indicadores por buffer.

In [47]:
barrios_shp_dir = "shp/divisiones/barrios_censo_2010"

In [48]:
recalculate_indicators.recalculate_indicators(barrios_shp_dir, "RADIO", skip=["14_10_2"])

Chequeamos que la cantidad de población calculada por barrio sea consistente con el total de la CABA.

In [49]:
shp_indic_barrios = "indicadores/barrios_censo_2010/barrios_censo_2010"
shp_indic_comunas = "indicadores/comunas_censo_2010/comunas_censo_2010"
shp_indic_fracciones = "indicadores/fracciones_censo_2010/fracciones_censo_2010"
shp_indic_radios = "indicadores/radios_censo_2010/radios_censo_2010"

In [50]:
def total_caba_population(shp_indicators):
    sf = shapefile.Reader(shp_indicators)
    pop_index = [field[0] for field in sf.fields[1:]].index("hab")
    return sum((record[pop_index] for record in sf.iterRecords() if record[pop_index] is not None))

In [51]:
print total_caba_population(shp_indic_barrios)
print total_caba_population(shp_indic_comunas)
print total_caba_population(shp_indic_fracciones)
print total_caba_population(shp_indic_radios)

print "Margen de error", total_caba_population(shp_indic_barrios) / total_caba_population(shp_indic_comunas) - 1

2889007.41538
2890151.0
2890151.0
2890151.0
Margen de error -0.000395683345081
